# 导入环境

In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [2]:
# 将JSON文件转换为CSV文件
df = pd.read_json('dataset/example.json')
ds = Dataset.from_pandas(df)

In [3]:
ds[:3]

{'instruction': ['解析命令行参数用于训练分割模型是怎样运作的？',
  '解析命令行参数用于训练分割模型是什么达成的？',
  '解析命令行参数用于训练分割模型是如何完成的？'],
 'input': ["当前代码仓是一个Python项目，实现了对分割模型的训练。包含以下函数：[{'name': 'parse_args', 'code': '', 'file': 'python\\\\train.py', 'metadata': {'description': '解析命令行参数用于训练分割模型', 'logic_steps': ['1. 创建参数解析器并设置描述信息', '2. 添加必须的配置文件路径参数', '3. 添加可选的工作目录参数', '4. 添加是否从检查点恢复的选项', '5. 添加是否启用混合精度训练的选项', '6. 添加配置覆盖选项，允许修改配置文件中的特定设置', '7. 添加任务启动器类型选项', '8. 添加本地排名参数，支持分布式训练', '9. 解析输入的命令行参数', '10. 设置环境变量中的本地排名信息', '11. 返回解析后的所有参数']}}, {'name': 'main', 'code': '', 'file': 'python\\\\train.py', 'metadata': {'description': '根据输入参数配置并启动训练过程', 'logic_steps': ['1. 解析输入参数', '2. 加载配置文件并根据参数调整配置', '3. 处理工作目录设置', '4. 配置自动混合精度训练选项', '5. 设置是否从检查点恢复训练', '6. 根据配置选择并初始化训练器', '7. 启动训练过程']}}]",
  "当前代码仓是一个Python项目，实现了对分割模型的训练。包含以下函数：[{'name': 'parse_args', 'code': '', 'file': 'python\\\\train.py', 'metadata': {'description': '解析命令行参数用于训练分割模型', 'logic_steps': ['1. 创建参数解析器并设置描述信息', '2. 添加必须的配置文件路径参数', '3. 添加可选的工作目录参数', '4. 添加是否从检查点恢复的选

# 处理数据集

In [4]:
tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/qwen/Qwen2.5-7B-Instruct', use_fast=False, trust_remote_code=True)
tokenizer

Qwen2Tokenizer(name_or_path='/root/autodl-tmp/qwen/Qwen2.5-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, sing

In [5]:
def process_func(example):
    MAX_LENGTH = 3840    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n你是一个代码专家，请根据以下信息回答问题<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26
})

In [7]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n你是一个代码专家，请根据以下信息回答问题<|im_end|>\n<|im_start|>user\n解析命令行参数用于训练分割模型是怎样运作的？当前代码仓是一个Python项目，实现了对分割模型的训练。包含以下函数：[{\'name\': \'parse_args\', \'code\': \'\', \'file\': \'python\\\\train.py\', \'metadata\': {\'description\': \'解析命令行参数用于训练分割模型\', \'logic_steps\': [\'1. 创建参数解析器并设置描述信息\', \'2. 添加必须的配置文件路径参数\', \'3. 添加可选的工作目录参数\', \'4. 添加是否从检查点恢复的选项\', \'5. 添加是否启用混合精度训练的选项\', \'6. 添加配置覆盖选项，允许修改配置文件中的特定设置\', \'7. 添加任务启动器类型选项\', \'8. 添加本地排名参数，支持分布式训练\', \'9. 解析输入的命令行参数\', \'10. 设置环境变量中的本地排名信息\', \'11. 返回解析后的所有参数\']}}, {\'name\': \'main\', \'code\': \'\', \'file\': \'python\\\\train.py\', \'metadata\': {\'description\': \'根据输入参数配置并启动训练过程\', \'logic_steps\': [\'1. 解析输入参数\', \'2. 加载配置文件并根据参数调整配置\', \'3. 处理工作目录设置\', \'4. 配置自动混合精度训练选项\', \'5. 设置是否从检查点恢复训练\', \'6. 根据配置选择并初始化训练器\', \'7. 启动训练过程\']}}]<|im_end|>\n<|im_start|>assistant\n解析命令行参数用于训练分割模型的实现方式：\n• 实现函数：parse_args\n• 文件位置：python\\train.py\n• 实现步骤：\n  - 1. 创建参数解析器并设置描述信息\n  - 2. 添加必须的配置文件路径参数\n  - 3. 添加可选的工作目录参数\n  - 4. 添加是否从检

In [8]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'解析命令行参数用于训练分割模型的实现方式：\n• 实现函数：parse_args\n• 文件位置：python\\train.py\n• 实现步骤：\n  - 1. 创建参数解析器并设置描述信息\n  - 2. 添加必须的配置文件路径参数\n  - 3. 添加可选的工作目录参数\n  - 4. 添加是否从检查点恢复的选项\n  - 5. 添加是否启用混合精度训练的选项\n  - 6. 添加配置覆盖选项，允许修改配置文件中的特定设置\n  - 7. 添加任务启动器类型选项\n  - 8. 添加本地排名参数，支持分布式训练\n  - 9. 解析输入的命令行参数\n  - 10. 设置环境变量中的本地排名信息\n  - 11. 返回解析后的所有参数\n• 完整代码：\n```python\ndef parse_args():\n    parser = argparse.ArgumentParser(description=\'Train a segmentor\')\n    parser.add_argument(\'config\', help=\'train config file path\')\n    parser.add_argument(\'--work-dir\', help=\'the dir to save logs and models\')\n    parser.add_argument(\'--resume\', action=\'store_true\', default=False, help=\'resume from the latest checkpoint in the work_dir automatically\')\n    parser.add_argument(\'--amp\', action=\'store_true\', default=False, help=\'enable automatic-mixed-precision training\')\n    parser.add_argument(\'--cfg-options\', nargs=\'+\', action=DictAction, help=\'override some settings in the used config, the k

# 创建模型

In [2]:
import torch

model = AutoModelForCausalLM.from_pretrained('/root/autodl-tmp/qwen/Qwen2.5-7B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm):

In [3]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [4]:
model.dtype

torch.bfloat16

# lora 

In [5]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'down_proj', 'o_proj', 'up_proj', 'k_proj', 'gate_proj', 'v_proj', 'q_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [6]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/root/autodl-tmp/qwen/Qwen2.5-7B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'down_proj', 'o_proj', 'up_proj', 'k_proj', 'gate_proj', 'v_proj', 'q_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [7]:
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.26434798934534914


# 配置训练参数

In [15]:
args = TrainingArguments(
    output_dir="./output/example",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=100,
    save_steps=100, 
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [16]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [17]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/root/miniconda3/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/autodl-tmp/qwen/Qwen2.5-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autoca

TrainOutput(global_step=300, training_loss=0.014179153277849159, metrics={'train_runtime': 818.1081, 'train_samples_per_second': 3.178, 'train_steps_per_second': 0.367, 'total_flos': 7.929096117291418e+16, 'train_loss': 0.014179153277849159, 'epoch': 92.3076923076923})

# 推理

## 场景1提问

In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

model_path = '/root/autodl-tmp/qwen/Qwen2.5-7B-Instruct/'
lora_path = './output/example/checkpoint-300'
input_content = "当前代码仓是一个Python项目，实现了对分割模型的训练。包含以下函数：[{'name': 'parse_args', 'code': '', 'file': 'python\\train.py', 'metadata': {'description': '解析命令行参数用于训练分割模型', 'logic_steps': ['1. 创建参数解析器并设置描述信息', '2. 添加必须的配置文件路径参数', '3. 添加可选的工作目录参数', '4. 添加是否从检查点恢复的选项', '5. 添加是否启用混合精度训练的选项', '6. 添加配置覆盖选项，允许修改配置文件中的特定设置', '7. 添加任务启动器类型选项', '8. 添加本地排名参数，支持分布式训练', '9. 解析输入的命令行参数', '10. 设置环境变量中的本地排名信息', '11. 返回解析后的所有参数']}}, {'name': 'main', 'code': '', 'file': 'python\\train.py', 'metadata': {'description': '根据输入参数配置并启动训练过程', 'logic_steps': ['1. 解析输入参数', '2. 加载配置文件并根据参数调整配置', '3. 处理工作目录设置', '4. 配置自动混合精度训练选项', '5. 设置是否从检查点恢复训练', '6. 根据配置选择并初始化训练器', '7. 启动训练过程']}}]"
# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = "解析命令行参数用于训练分割模型是如何完成的？"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "你是一个代码专家，请根据以下信息回答问题"},
                                        {"role": "user", "content": input_content},
                                        {"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cpu')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}

with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

解析命令行参数用于训练分割模型的实现方式：
• 实现函数：parse_args
• 文件位置：python\train.py
• 实现步骤：
  - 1. 创建参数解析器并设置描述信息
  - 2. 添加必须的配置文件路径参数
  - 3. 添加可选的工作目录参数
  - 4. 添加是否从检查点恢复的选项
  - 5. 添加是否启用混合精度训练的选项
  - 6. 添加配置覆盖选项，允许修改配置文件中的特定设置
  - 7. 添加任务启动器类型选项
  - 8. 添加本地排名参数，支持分布式训练
  - 9. 解析输入的命令行参数
  - 10. 设置环境变量中的本地排名信息
  - 11. 返回解析后的所有参数


## 场景2提问

In [20]:
prompt = "如何增加训练完成后的自动模型评估功能"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "你是一个代码专家，请根据以下信息回答问题"},
                                        {"role": "user", "content": input_content},
                                        {"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cpu')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}

with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

要实现训练完成后自动进行模型评估的功能，可以在训练结束后添加模型评估步骤。这通常包括加载测试数据集、计算模型在测试集上的性能指标（如准确率、损失值等）。实现这一功能的步骤如下：

  * 1. 在训练结束后添加模型评估的步骤；
  * 2. 加载测试数据集；
  * 3. 计算模型在测试集上的性能指标；
  * 4. 将评估结果保存或显示给用户。

实现方式：
  * 通过添加命令行参数来控制是否执行评估步骤；
  * 实现评估功能的函数；
  * 在训练结束后调用评估函数；
  * 完成后显示或保存评估结果。
